<a href="https://colab.research.google.com/github/usrmaia/Mundo-de-Wumpus/blob/main/Mundo_de_Wumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from random import random, randint, choice
!pip install tabulate # Exibe dados em uma grid
from tabulate import tabulate
!pip install colorama # Dar cor ao texto
from colorama import Fore, init, deinit
!pip install python-sat
from pysat.solvers import Glucose3
!pip install db-sqlite3
import sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 13.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 171 kB 13.1 MB/s 
  Created wheel for db-sqlite3: filename=db_sqlite3-0.0.1-py3-none-any.whl size=1794 sha256=d0a2765426fda51e3609ef319f8a3bbdecea79de6f09a41cf1f4e7e5867b31b0
  Stored in directory: /root/.cache/pip/wheels/02/38/d5/2f54461050571bf5330fee2a37ab1c9b5e7540b0572f1acdab
  Created wheel for db: filename=db-0.1.1-py3-none-any.whl size=3895 sha256=94edc2afe36e4bf4f0a9e262cc27ca0298cd6d931666271cfe3560cdcaa230d3
  Stored in directory: /root/.cache/pip/wheels/8e/97/82/741d2b360507411ec233d0280d7

# Base de Conhecimento

In [2]:
class BaseDeConhecimento:
  def __init__(self):
    self.conn = sqlite3.connect("base_de_conhecimento")
    self.cursor = self.conn.cursor()
    self.executarComando(self, """
      create table if not exists base_de_conhecimento (
        tempo integer primary key autoincrement,
        conhecimento varchar,
        raciocinio varchar,
        acao varchar
      )
    """)
  
  @staticmethod
  def executarComando(self, comando):
    try:
      self.cursor.execute(comando)
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
    
  def imprimirTabela(self):
    try:
      self.cursor.execute(f"select * from base_de_conhecimento")
      sentencas = self.cursor.fetchall()
      header = ["Tempo", "Conhecimento", "Raciocínio", "Ação"]
      print(tabulate(sentencas, tablefmt="simple", stralign="center", headers=header))
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def retornarTabela(self):
    try:
      self.cursor.execute(f"select * from conhecimento")
      tabela = self.cursor.fetchall()
      return tabela
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

  def inserirLinha(self):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (conhecimento, raciocinio, acao) values ('', '', '')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def inserirConhecimento(self, conhecimento):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (conhecimento) values ('{conhecimento}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def inserirRaciocinio(self, raciocinio):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (raciocinio) values ('{raciocinio}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def inserirAcao(self, acao):
    try:
      self.cursor.execute(f"insert into base_de_conhecimento (acao) values ('{acao}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

# Geração do Mapa
Para cada posição temos **((wumpus, poco, ouro, parede), (fedor, brisa, brilho, impacto, grito))** atribuído de forma aleatória.

Esse dado (tupla) será captado pelo sensor.
### Exemplos:
* Posição segura: ((0, 0, 0, 0), (0, 0, _, _, _))
* Posição com 3 poços adjacentes: ((0, 0, 0, 0), (_, 3, _, _, _))

## Declaração
mapa = Mapa(linhas, colunas, quantidade_wumpus, quantidade_pocos, quantidade_ouros)

In [3]:
class Mapa():
  def __init__(self, linhas, colunas, quantidade_wumpus, quantidade_pocos, quantidade_ouros):
    self.linhas = linhas
    self.colunas = colunas
    self.quantidade_wumpus = quantidade_wumpus
    self.quantidade_pocos = quantidade_pocos
    self.quantidade_ouros = quantidade_ouros
    self.taxa_wumpus = self.quantidade_wumpus / (self.linhas * self.colunas - 1 - self.quantidade_pocos - self.quantidade_ouros)
    self.taxa_pocos = self.quantidade_pocos / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_ouros)
    self.taxa_ouros = self.quantidade_ouros / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_pocos)
    self.status_posicao = {} # linha_coluna_posicao:((wumpus, poco, ouro), (fedor, brisa, brilho, impacto, grito))
    self.mapa = []
    self.mapearStatusDePosicao(self)

  @staticmethod
  def mapearStatusDePosicao(self):
    for linha in range(-1, self.linhas + 1):
      for coluna in range(-1, self.colunas + 1):
        self.status_posicao[f"{linha}_{coluna}_posicao"] = ((0, 0, 0, 0), (0, 0, 0, 0, 0))
  
  def gerarMapa(self):
    for linha in range(-1, self.linhas + 1):
      for coluna in range(-1, self.colunas + 1):
        if self.insercaoValida(self, linha, coluna):
          self.adicionarWumpusEFedor(self, linha, coluna)
          self.adicionarPocoEBrisa(self, linha, coluna)
          self.adicionarOuroEBrilho(self, linha, coluna)
        self.adicionarParedeEImpacto(self, linha, coluna)

  def imprimirMapa(self):
    self.mapa = []
    init(convert=True, autoreset=True) # colorama
    for linha in range(-1, self.linhas + 1):
      elementos_da_linha = []
      for coluna in range(-1, self.colunas + 1):
        elementos_da_celula = ""
        if self.possuiParede(self, linha, coluna):
          elementos_da_celula += (Fore.MAGENTA + "parede")
        if self.possuiImpacto(self, linha, coluna):
          elementos_da_celula += (Fore.MAGENTA + str(self.possuiImpacto(self, linha, coluna)) + "impacto")
        if self.possuiWumpus(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + "wumpus")
        if self.possuiPoco(self, linha, coluna):
          elementos_da_celula += (Fore.RED + "poco")
        if self.possuiOuro(self, linha, coluna):
          elementos_da_celula += (Fore.YELLOW + "ouro")
        if self.possuiFedor(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + str(self.possuiFedor(self, linha, coluna)) + "fedor")
        if self.possuiBrisa(self, linha, coluna):
          elementos_da_celula += (Fore.RED + str(self.possuiBrisa(self, linha, coluna)) + "brisa")
        if self.possuiBrilho(self, linha, coluna):
          elementos_da_celula += (Fore.YELLOW + "brilho")
        if self.possuiOK(self, linha, coluna):
          elementos_da_celula += (Fore.WHITE + "ok")
      
        elementos_da_linha.append(elementos_da_celula)
      self.mapa.append(elementos_da_linha)
    deinit() # colorama
    print(Fore.WHITE) # gambiarra
    print(tabulate(self.mapa, tablefmt="plain", stralign="center"))
    print(Fore.WHITE) # gambiarra
  
  def retornarStatusDaPosica(self, linha, coluna):
    try:
      return self.status_posicao[f"{linha}_{coluna}_posicao"]
    except:
      print("Posição inválida!")
      return False

  def pegarOuro(self, linha, coluna):
    ouro = self.status_posicao[f"{linha}_{coluna}_posicao"][0][2]
    if ouro:
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0][0], status[0][1], status[0][2] - 1), (status[1][0], status[1][1], status[1][2] - 1, status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status
      return True
    return False

  @staticmethod
  def insercaoValida(self, linha, coluna):
    return not ((linha == 0 and coluna == 0) or (linha == -1 or linha == self.linhas or coluna == -1 or coluna == self.colunas))

  @staticmethod
  def adicionarWumpusEFedor(self, linha, coluna):
    if self.quantidade_wumpus >= 1:
      if random() < self.taxa_wumpus:
        self.adicionarWumpus(self, linha, coluna)
        self.adicionarFedor(self, linha - 1, coluna)
        self.adicionarFedor(self, linha, coluna - 1)
        self.adicionarFedor(self, linha + 1, coluna)
        self.adicionarFedor(self, linha, coluna + 1)
        self.quantidade_wumpus -= 1
  
  @staticmethod
  def adicionarPocoEBrisa(self, linha, coluna):
    if self.quantidade_pocos >= 1 and not self.possuiWumpus(self, linha, coluna):
      if random() < self.taxa_pocos:
        self.adicionarPoco(self, linha, coluna)
        self.adicionarBrisa(self, linha - 1, coluna)
        self.adicionarBrisa(self, linha, coluna - 1)
        self.adicionarBrisa(self, linha + 1, coluna)
        self.adicionarBrisa(self, linha, coluna + 1)
        self.quantidade_pocos -= 1
  
  @staticmethod
  def adicionarOuroEBrilho(self, linha, coluna):
    if self.quantidade_ouros >= 1 and not (self.possuiWumpus(self, linha, coluna) or self.possuiPoco(self, linha, coluna)):
      if random() < self.taxa_ouros:
        self.adicionarOuro(self, linha, coluna)
        self.adicionarBrilho(self, linha, coluna)
        self.quantidade_ouros -= 1
  
  @staticmethod
  def adicionarParedeEImpacto(self, linha, coluna):
    if (linha == -1 or linha == self.linhas) or (coluna == -1 or coluna == self.colunas):
      self.adicionarParede(self, linha, coluna)
    else:
      if (linha == 0 or linha == self.linhas - 1):
        self.adicionarImpacto(self, linha, coluna)
      if (coluna == 0 or coluna == self.colunas - 1):
        self.adicionarImpacto(self, linha, coluna)

  @staticmethod
  def adicionarWumpus(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0] + 1, status[0][1], status[0][2], status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarPoco(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1] + 1, status[0][2], status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarOuro(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1], status[0][2] + 1, status[0][3]), (status[1]))
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarParede(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0][0], status[0][1], status[0][2], status[0][3] + 1), (status[1])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status

  @staticmethod
  def adicionarFedor(self, linha, coluna):
    if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0]), (status[1][0] + 1, status[1][1], status[1][2], status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status

  @staticmethod
  def adicionarBrisa(self, linha, coluna):
    if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
      status = self.status_posicao[f"{linha}_{coluna}_posicao"]
      status = ((status[0]), (status[1][0], status[1][1] + 1, status[1][2], status[1][3], status[1][4]))
      self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarBrilho(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2] + 1, status[1][3], status[1][4])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def adicionarImpacto(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2], status[1][3] + 1, status[1][4])) 
    self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
  @staticmethod
  def possuiWumpus(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][0]:
      return True
    return False
  
  @staticmethod
  def possuiPoco(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][1]:
      return True
    return False
  
  @staticmethod
  def possuiOuro(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][2]:
      return True
    return False
  
  @staticmethod
  def possuiParede(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][0][3]:
      return True
    return False
  
  @staticmethod
  def possuiFedor(self, linha, coluna):
    fedor = self.status_posicao[f"{linha}_{coluna}_posicao"][1][0]
    if fedor:
      return fedor
    return False
  
  @staticmethod
  def possuiBrisa(self, linha, coluna):
    brisa = self.status_posicao[f"{linha}_{coluna}_posicao"][1][1]
    if brisa:
      return brisa
    return False
  
  @staticmethod
  def possuiBrilho(self, linha, coluna):
    if self.status_posicao[f"{linha}_{coluna}_posicao"][1][2]:
      return True
    return False
  
  @staticmethod
  def possuiImpacto(self, linha, coluna):
    impacto = self.status_posicao[f"{linha}_{coluna}_posicao"][1][3]
    if impacto:
      return impacto
    return False
  
  @staticmethod
  def possuiOK(self, linha, coluna):
    status = self.status_posicao[f"{linha}_{coluna}_posicao"]
    for tupla in status:
      for elemento in tupla:
        if elemento != 0:
          return False
    return True

In [4]:
mapa = Mapa(4, 4, 1, 2, 2)
mapa.gerarMapa()
mapa.imprimirMapa()


parede         parede               parede            parede       parede   parede
parede  2impacto1fedor1brisa     1impactopoco     1impacto1brisa  2impacto  parede
parede     1impactowumpus        1fedor1brisa           ok        1impacto  parede
parede     1impacto1fedor             ok                ok        1impacto  parede
parede        2impacto        1impactoourobrilho     1impacto     2impacto  parede
parede         parede               parede            parede       parede   parede



# Geração do Mapa do Conhecimento
Mapa gerado a partir da base de conhecimento (linguagem natural)

In [5]:
class MapaConhecimento():
    def __init__(self):
        self.conhecimento = []
        self.mapa = []
        self.linhas = 0
        self.colunas = 0
    
    def inserirConhecimento(self, conhecimento):
        self.conhecimento = conhecimento
        self.descobrirLinhaEColunaMaximo(self)
        
    def imprimirMapa(self):
      init() # colorama
      for linha in range(0, self.linhas):
        elementos_da_linha = []
        for coluna in range(0, self.colunas):
          elementos_da_celula = ""
          if f"{linha}_{coluna}_wumpus" in self.conhecimento:
            elementos_da_celula += (Fore.GREEN + "wumpus")
          if f"{linha}_{coluna}_poco" in self.conhecimento:
            elementos_da_celula += (Fore.RED + "poco")
          if f"{linha}_{coluna}_ouro" in self.conhecimento:
            elementos_da_celula += (Fore.YELLOW + "ouro")
          if f"{linha}_{coluna}_fedor" in self.conhecimento:
            elementos_da_celula += (Fore.GREEN + "fedor")
          if f"{linha}_{coluna}_brisa" in self.conhecimento:
            elementos_da_celula += (Fore.RED + "brisa")
          if f"{linha}_{coluna}_brilho" in self.conhecimento:
            elementos_da_celula += (Fore.YELLOW + "brilho")
          if f"{linha}_{coluna}_impacto" in self.conhecimento:
            elementos_da_celula += (Fore.BLACK + "parede")
        self.mapa.append(elementos_da_linha)
      deinit() # colorama
      print(tabulate(self.mapa, tablefmt="plain", stralign="center"))
      print(Fore.WHITE) # gambiarra
    
    @staticmethod
    def descobrirLinhaEColunaMaximo(self):
        conhecimento = self.conhecimento
        linhas = 0
        colunas = 0
        for instancia in conhecimento:
          linha, coluna = self.retornarLinhaEColuna(self, instancia)
          if linha > self.linhas: self.linhas = linha
          if coluna > self.coluna: self.coluna = coluna

    @staticmethod
    def retornarLinhaEColuna(self, instancia):
        index = 0
        linha = ""
        while(instancia[index] != "_"):
            linha += instancia[index]
            index += 1

        index += 1
        coluna = ""
        while(instancia[index] != "_"):
            coluna += instancia[index]
            index += 1

        linha = int(linha)
        coluna = int(coluna)
        return linha, coluna

# Modelagem do Mundo de Wumpus para Solver 👾
```
mapeamento = {
  '0_0_wumpus': 0,
  '0_0_poco': 1, 
  '0_0_ouro': 2, 
  '0_0_fedor': 3, 
  '0_0_brisa': 4, 
  '0_0_brilho': 5,
  '0_0_impacto': 6,
  '0_0_grito': 7,
  ...
  'verdade': index
  'falso': -index
}

mapeamento_reverso = {
  0: '0_0_wumpus',
  1: '0_0_poco',
  2: '0_0_ouro',
  3: '0_0_fedor', 
  4: '0_0_brisa',
  5: '0_0_brilho',
  6: '0_0_impacto',
  7: '0_0_grito',
  ...
  index: 'verdade'
  -index: 'falso'
}
```

# Equivalências Lógicas do Mundo de Wumpus Para FNC - Solver 👾
## 0 Wumpus
### 1 regra:
1. pos_wumpus>F => fnc-solve => -pos_wumpus

## 1 Wumpus

## 0 Poço
### 1 regra:
1. pos_poco>F => fnc-solve => -pos_poco

## 1 Poço

## 0 Ouro
### 1 regra:
1. pos_ouro>F => fnc-solve => -pos_ouro

## 1 Ouro
### 1 regra:
1. pos_ouro>V => fnc-solve => pos_ouro
2. pos_ouro > pos_brilho => fnc-solve => -pos_ouro#pos_brilho

## 0 Fedor/Brisa
### 2 regras:
1. pos_>F => fnc-solve => -pos_
2. -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)

## 1 Fedor/Brisa
### Uma única proposição genérica:
pos_>(c&-e&-b&-d)#(-c&e&-b&-d)#(-c&-e&b&-d)#(-c&-e&-b&d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
3. -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
4. -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
5. -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-e#-b)

## 2 Fedor/Brisa
### Uma única proposição genérica:
pos>(c&e)#(c&b)#(c&d)#(e&b)#(e&d)#(b&d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
3. -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
4. -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
5. -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)

## 3 Fedor/Brisa
### Uma única proposição genérica:
pos > (-c&e&b&d)#(c&-e&b&d)#(c&e&-b&d)#(c&e&b&-d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
3. -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
4. -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
5. -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)

## 0 Brilho
### 2 regra:
1. pos_brilho>F => fnc-solve => -pos_brilho
2. -pos_brilho>-pos_ouro => fnc-solve => (pos_brilho#-pos_ouro)

## 1 Brilho
### 2 regra:
1. pos_brilho>V => fnc-solve => pos_brilho
2. pos_brilho>pos_ouro => fnc-solve => (-pos_brilho#pos_ouro)

## 0 Parede
### 1 regra:
1. pos_parede>F => fnc-solve => -pos_parede

## 1 Parede
### 1 regra:
1. pos_parede>V => fnc-solve => pos_parede

## 1 OK
### 1 regra:
1. pos_ok>V => fnc-solve => pos_ok

## 0 Impacto
### 3 regra:
1. pos_impacto>F => fnc-solver => -pos_impacto
2. -pos_impacto>-pos_parede => fnc-solver => pos_impacto#-pos_parede
3. -pos_impacto>-(c#e#b#d) => fnc-solver => (pos_impacto#-c)&(pos_impacto#-e)&(pos_impacto#-b)&(pos_impacto#-d)

## 1 Impacto
### 5 regra:
1. pos_impacto>V => fnc-solver => pos_impacto
2. -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
3. -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
4. -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
5. -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-e#-b)
6. pos_impacto&c_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-c#e#d)&(-p#-c#-d#-e)
7. pos_impacto&b_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-b#e#d)&(-p#-b#-d#-e)
8. pos_impacto&e_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-e#c#b)&(-p#-e#-b#-c)
9. pos_impacto&d_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-d#c#b)&(-p#-d#-c#-b)
10. c>(-e&-b&-d) => fnc-solve => (-c#-e)&(-c#-b)&(-c#-d)
11. e>(-c&-b&-d) => fnc-solve => (-e#-c)&(-e#-b)&(-e#-d)
12. b>(-c&-e&-d) => fnc-solve => (-b#-c)&(-b#-e)&(-b#-d)
13. d>(-c&-e&-b) => fnc-solve => (-d#-c)&(-d#-e)&(-d#-b)

## 2 Impacto
### 5 regra:
1. pos_impacto>V => fnc-solver => pos_impacto
2. -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
3. -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
4. -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
5. -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
6. pos_impacto&c_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-c#e#d)&(-p#-c#-d#-e)
7. pos_impacto&b_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-b#e#d)&(-p#-b#-d#-e)
8. pos_impacto&e_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-e#c#b)&(-p#-e#-b#-c)
9. pos_impacto&d_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-d#c#b)&(-p#-d#-c#-b)
10. c>(-e&-b&-d) => fnc-solve => (-c#-e)&(-c#-b)&(-c#-d)
11. e>(-c&-b&-d) => fnc-solve => (-e#-c)&(-e#-b)&(-e#-d)
12. b>(-c&-e&-d) => fnc-solve => (-b#-c)&(-b#-e)&(-b#-d)
13. d>(-c&-e&-b) => fnc-solve => (-d#-c)&(-d#-e)&(-d#-b)

## 0 Grito
## 1 Grito

In [13]:
from os import stat
class Solver():
  def __init__(self, linhas, colunas, base_de_conhecimento):
    self.glucose = Glucose3()
    self.linhas = linhas
    self.colunas = colunas
    self.base_de_conhecimento = base_de_conhecimento
    self.mapeamento = {}
    self.mapeamento_reverso = {}
    self.mapear(self)
    self.glucose.add_clause([self.mapeamento[f"-1_0_parede"]])
    self.glucose.add_clause([self.mapeamento[f"0_-1_parede"]])
    self.glucose.add_clause([-self.mapeamento[f"1_0_parede"]])
    self.glucose.add_clause([-self.mapeamento[f"0_1_parede"]])
    self.atomo_cima = ""
    self.atomo_esquerda = ""
    self.atomo_direita = ""
    self.atomo_baixo = ""
  
  @staticmethod
  def mapear(self):
    index = 1
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        self.mapeamento[f"{linha}_{coluna}_wumpus"] = index
        self.mapeamento_reverso[index] = f"{linha}_{coluna}_wumpus"
        self.mapeamento_reverso[-index] = f"-{linha}_{coluna}_wumpus"

        self.mapeamento[f"{linha}_{coluna}_poco"] = index + 1
        self.mapeamento_reverso[index + 1] = f"{linha}_{coluna}_poco"
        self.mapeamento_reverso[-index - 1] = f"-{linha}_{coluna}_poco"

        self.mapeamento[f"{linha}_{coluna}_ouro"] = index + 2
        self.mapeamento_reverso[index + 2] = f"{linha}_{coluna}_ouro"
        self.mapeamento_reverso[-index - 2] = f"-{linha}_{coluna}_ouro"

        self.mapeamento[f"{linha}_{coluna}_parede"] = index + 3
        self.mapeamento_reverso[index + 3] = f"{linha}_{coluna}_ouro"
        self.mapeamento_reverso[-index - 3] = f"-{linha}_{coluna}_ouro"

        self.mapeamento[f"{linha}_{coluna}_fedor"] = index + 4
        self.mapeamento_reverso[index + 4] = f"{linha}_{coluna}_fedor"
        self.mapeamento_reverso[-index - 4] = f"-{linha}_{coluna}_fedor"

        self.mapeamento[f"{linha}_{coluna}_brisa"] = index + 5
        self.mapeamento_reverso[index + 5] = f"{linha}_{coluna}_brisa"
        self.mapeamento_reverso[-index - 5] = f"-{linha}_{coluna}_brisa"

        self.mapeamento[f"{linha}_{coluna}_brilho"] = index + 6
        self.mapeamento_reverso[index + 6] = f"{linha}_{coluna}_brilho"
        self.mapeamento_reverso[-index - 6] = f"-{linha}_{coluna}_brilho"

        self.mapeamento[f"{linha}_{coluna}_impacto"] = index + 7
        self.mapeamento_reverso[index + 7] = f"{linha}_{coluna}_impacto"
        self.mapeamento_reverso[-index - 7] = f"-{linha}_{coluna}_impacto"

        index += 8

    for linha in [-1, self.linhas]:
      for coluna in range(-1, self.colunas + 1):
        self.mapeamento[f"{linha}_{coluna}_parede"] = index
        self.mapeamento_reverso[index] = f"{linha}_{coluna}_parede"
        self.mapeamento_reverso[-index] = f"-{linha}_{coluna}_parede"
        
        index += 1
    
    for coluna in [-1, self.colunas]:
      for linha in range(-1, self.linhas + 1):
        self.mapeamento[f"{linha}_{coluna}_parede"] = index
        self.mapeamento_reverso[index] = f"{linha}_{coluna}_parede"
        self.mapeamento_reverso[-index] = f"-{linha}_{coluna}_parede"
        
        index += 1

    self.mapeamento[f"verdade"] = index
    self.mapeamento_reverso[index] = f"verdade"

    self.glucose.add_clause([index])

    self.mapeamento[f"falso"] = -index
    self.mapeamento_reverso[-index] = f"falso"

  def imprimirMapeamento(self):
    print("mapeamento[posição] -> index")
    print(self.mapeamento)
    print("mapeamento_reverso[index] -> posição")
    print(self.mapeamento_reverso)
  
  def perguntarRegra(self, regra):
    regra = self.mapeamento[regra]
    if self.glucose.solve(assumptions=[regra]): return True
    return False
  
  @staticmethod
  def inserirConhecimento(self, conhecimento):
    self.base_de_conhecimento.inserirConhecimento(f"{self.mapeamento_reverso[conhecimento]}")
  
  @staticmethod
  def inserirRaciocinio(self, raciocinio):
    regra = ""
    for atomo in raciocinio:
      regra += f"{self.mapeamento_reverso[atomo]}, "
    regra = regra[0:-2]
    self.base_de_conhecimento.inserirRaciocinio(regra)
  
  def resolverSolve(self):
    self.glucose.solve()

  def retornarSolve(self):
    self.glucose.get_model()
  
  def imprimirSolve(self):
    print(self.glucose.get_model())

  # Equivalências Lógicas

  @staticmethod
  def eVerdade(self):
    self.glucose.add_clause([self.atomo])

    self.base_de_conhecimento.inserirConhecimento(self.atomo)

  @staticmethod
  def eFalso(self):
    self.glucose.add_clause(-self.atomo)

    self.base_de_conhecimento.inserirConhecimento(-self.atomo)
  
  @staticmethod
  def naoPossui(self):
    self.glucose.add_clause([-self.atomo_cima])
    self.glucose.add_clause([-self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita])

    self.base_de_conhecimento.inserirConhecimento(-self.atomo_cima)
    self.base_de_conhecimento.inserirConhecimento(-self.atomo_esquerda)
    self.base_de_conhecimento.inserirConhecimento(-self.atomo_baixo)
    self.base_de_conhecimento.inserirConhecimento(-self.atomo_direita)

  @staticmethod
  def possui1Verdadeiro(self):
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_direita])

    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_esquerda])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_esquerda, -self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_esquerda, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_baixo, -self.atomo_direita])
  
  @staticmethod
  def possui2Verdadeiro(self):
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

  @staticmethod
  def possui3Verdadeiro(self):
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_direita])

    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_cima, self.atomo_esquerda])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_cima, self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_cima, self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_esquerda, self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_esquerda, self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([self.atomo_baixo, self.atomo_direita])
  
  @staticmethod
  def impactosAdjacentes(self):
    self.glucose.add_clause([-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

    self.glucose.add_clause([-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])   

    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.base_de_conhecimento.inserirRaciocinio([-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])   


  def naoPossuiWumpus(self, linha, coluna): 
    # pos_wumpus>F => fnc-solve => -pos_wumpus
    atomo = self.mapeamento[f"{linha}_{coluna}_wumpus"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

  def possuiWumpus(self, linha, coluna): pass

  def naoPossuiPoco(self, linha, coluna):
    # pos_poco>F => fnc-solve => -pos_poco
    atomo = self.mapeamento[f"{linha}_{coluna}_poco"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)
  
  def possuiPoco(self, linha, coluna): pass

  def naoPossuiOuro(self, linha, coluna):
    # pos_ouro>F => fnc-solve => -pos_ouro
    atomo = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

  def possuiOuro(self, linha, coluna):
    # pos_ouro>V => fnc-solve => pos_ouro
    atomo_1 = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.glucose.add_clause([atomo_1])

    # pos_ouro>pos_brilho => fnc-solve => -pos_ouro#pos_brilho
    atomo_2 = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.glucose.add_clause([-atomo_1, atomo_2])

    self.inserirConhecimento(self, -atomo_1)
    self.inserirRaciocinio(self, [-atomo_1, atomo_2])
  
  def naoPossuiParede(): pass
  def possuiParede(): pass
  
  def naoPossuiFedor(self, linha, coluna):
    # pos_>F => fnc-solve => -pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

    # -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)
    try:
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_wumpus"]
      self.glucose.add_clause([atomo, -self.atomo_cima])

      self.inserirRaciocinio(self, [atomo, -self.atomo_cima])
    except: pass

    try:
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_wumpus"]
      self.glucose.add_clause([atomo, -self.atomo_esquerda])

      self.inserirRaciocinio(self, [atomo, -self.atomo_esquerda])
    except: pass

    try:
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_wumpus"]
      self.glucose.add_clause([atomo, -self.atomo_baixo])

      self.inserirRaciocinio(self, [atomo, -self.atomo_baixo])
    except: pass

    try:
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_wumpus"]
      self.glucose.add_clause([atomo, -self.atomo_direita])

      self.inserirRaciocinio(self, [atomo, -self.atomo_direita])
    except: pass

  def naoPossuiBrisa(self, linha, coluna):
    # pos_>F => fnc-solve => -pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

    # -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)
    try:
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_poco"]
      self.glucose.add_clause([atomo, -self.atomo_cima])

      self.inserirRaciocinio(self, [atomo, -self.atomo_cima])
    except: pass

    try:
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_poco"]
      self.glucose.add_clause([atomo, -self.atomo_esquerda])

      self.inserirRaciocinio(self, [atomo, -self.atomo_esquerda])
    except: pass

    try:
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_poco"]
      self.glucose.add_clause([atomo, -self.atomo_baixo])

      self.inserirRaciocinio(self, [atomo, -self.atomo_baixo])
    except: pass

    try:
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_poco"]
      self.glucose.add_clause([atomo, -self.atomo_direita])

      self.inserirRaciocinio(self, [atomo, -self.atomo_direita])
    except: pass

  def possui1Fedor(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_wumpus"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_wumpus"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_wumpus"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_wumpus"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])

    # -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    # -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    # -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-e#-b)
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])
    

  def possui1Brisa(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_poco"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_poco"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_poco"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_poco"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])

    # -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    # -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    # -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-b#-d)
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])
     
  def possui2Fedor(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_wumpus"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_wumpus"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_wumpus"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_wumpus"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    # -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    # -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    # -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

  def possui2Brisa(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_poco"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_poco"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_poco"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_poco"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    # -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    # -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    # -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

  def possui3Fedor(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_wumpus"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_wumpus"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_wumpus"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_wumpus"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_direita])

    # -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_direita])

    # -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_direita])

    # -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima])
    self.glucose.add_clause([self.atomo_direita, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_baixo])
  
  def possui3Brisa(self, linha, coluna):
    # pos_>V => fnc-solve => pos_
    atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.glucose.add_clause([atomo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_poco"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_poco"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_poco"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_poco"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_direita])

    # -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_direita])

    # -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_direita])

    # -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima])
    self.glucose.add_clause([self.atomo_direita, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, self.atomo_baixo])

    self.inserirConhecimento(self, atomo)
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_baixo])

  def naoPossuiBrilho(self, linha, coluna):
    # pos_brilho>F => fnc-solve => -pos_brilho
    atomo_1 = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.glucose.add_clause([-atomo_1])

    # -pos_brilho>-pos_ouro => fnc-solve => (pos_brilho#-pos_ouro)
    atomo_2 = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.glucose.add_clause([atomo_1, -atomo_2])

    self.inserirConhecimento(self, -atomo_1)
    self.inserirRaciocinio(self, [atomo_1, -atomo_2])
  
  def possuiBrilho(self, linha, coluna):
    # pos_brilho>V => fnc-solve => pos_brilho
    atomo_1 = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.glucose.add_clause([atomo_1])

    # pos_brilho>pos_ouro => fnc-solve => (-pos_brilho#pos_ouro)
    atomo_2 = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.glucose.add_clause([-atomo_1, atomo_2])

    self.inserirConhecimento(self, atomo_1)
    self.inserirRaciocinio(self, [-atomo_1, atomo_2])
  
  def naoPossuiImpacto(self, linha, coluna):
    # pos_impacto>F => fnc-solver => -pos_impacto
    atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

    # -pos_impacto>(-c&-e&-b&-d) => fnc-solver => (pos_impacto#-c)&(pos_impacto#-e)&(pos_impacto#-b)&(pos_impacto#-d)
    try:
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
      self.glucose.add_clause([atomo, -self.atomo_cima])

      self.inserirRaciocinio(self, [atomo, -self.atomo_cima])
    except: pass

    try:
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
      self.glucose.add_clause([atomo, -self.atomo_esquerda])

      self.inserirRaciocinio(self, [atomo, -self.atomo_esquerda])
    except: pass

    try:
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
      self.glucose.add_clause([atomo, -self.atomo_baixo])

      self.inserirRaciocinio(self, [atomo, -self.atomo_baixo])
    except: pass

    try:
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
      self.glucose.add_clause([atomo, -self.atomo_direita])

      self.inserirRaciocinio(self, [atomo, -self.atomo_direita])
    except: pass

  def possui1Impacto(self, linha, coluna):
    
    atomo = self.mapeamento[f"{linha}_{coluna}_parede"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

    # pos_impacto>V => fnc-solver => pos_impacto
    atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.glucose.add_clause([atomo])

    self.inserirConhecimento(self, atomo)

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    
    # -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    # -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    # -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-e#-b)
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])

    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, -self.atomo_esquerda, -self.atomo_baixo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]
    except: self.atomo_cima = self.mapeamento[f"falso"]

    # pos_impacto&c_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-c#e#d)&(-p#-c#-d#-e)
    self.glucose.add_clause([-atomo, -self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-atomo, -self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]

    # pos_impacto&b_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-b#e#d)&(-p#-b#-d#-e)
    self.glucose.add_clause([-atomo, -self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-atomo, -self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]

    # pos_impacto&e_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-e#c#b)&(-p#-e#-c#-b)
    self.glucose.add_clause([-atomo, -self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-atomo, -self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # pos_impacto&d_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-d#c#b)&(-p#-d#-c#-b)
    self.glucose.add_clause([-atomo, -self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-atomo, -self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # c>(-e&-b&-d) => fnc-solve => (-c#-e)&(-c#-b)&(-c#-d)
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_direita])

    # e>(-c&-b&-d) => fnc-solve => (-e#-c)&(-e#-b)&(-e#-d)
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_cima])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_direita])

    # b>(-c&-e&-d) => fnc-solve => (-b#-c)&(-b#-e)&(-b#-d)
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_cima])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_direita])

    # d>(-c&-e&-b) => fnc-solve => (-d#-c)&(-d#-e)&(-d#-b)
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_cima])
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_baixo])

  def possui2Impacto(self, linha, coluna):
    atomo = self.mapeamento[f"{linha}_{coluna}_parede"]
    self.glucose.add_clause([-atomo])

    self.inserirConhecimento(self, -atomo)

    # pos_impacto>V => fnc-solver => pos_impacto
    atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.glucose.add_clause([atomo])

    self.inserirConhecimento(self, atomo)

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    # -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    # -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    # -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])

    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_direita, self.atomo_esquerda, self.atomo_baixo])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]
    except: self.atomo_cima = self.mapeamento[f"falso"]

    # pos_impacto&c_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-c#e#d)&(-p#-c#-d#-e)
    self.glucose.add_clause([-atomo, -self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-atomo, -self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]

    # pos_impacto&b_impacto>(e&-d)#(-e&d) => fnc-solve => (-p#-b#e#d)&(-p#-b#-d#-e)
    self.glucose.add_clause([-atomo, -self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-atomo, -self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: self.atomo_cima = self.mapeamento[f"falso"]
    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except: self.atomo_baixo = self.mapeamento[f"falso"]
    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]
    except: self.atomo_esquerda = self.mapeamento[f"falso"]

    # pos_impacto&e_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-e#c#b)&(-p#-e#-c#-b)
    self.glucose.add_clause([-atomo, -self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-atomo, -self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]
    except: self.atomo_direita = self.mapeamento[f"falso"]

    # pos_impacto&d_impacto>(c&-b)#(-c&b) => fnc-solve => (-p#-d#c#b)&(-p#-d#-c#-b)
    self.glucose.add_clause([-atomo, -self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-atomo, -self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

    self.inserirRaciocinio(self, [-atomo, -self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-atomo, -self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])

SyntaxError: ignored

In [7]:
base = BaseDeConhecimento()
solver = Solver(6, 6, base)
solver.imprimirMapeamento()

mapeamento[posição] -> index
{'0_0_wumpus': 1, '0_0_poco': 2, '0_0_ouro': 3, '0_0_parede': 4, '0_0_fedor': 5, '0_0_brisa': 6, '0_0_brilho': 7, '0_0_impacto': 8, '0_1_wumpus': 9, '0_1_poco': 10, '0_1_ouro': 11, '0_1_parede': 12, '0_1_fedor': 13, '0_1_brisa': 14, '0_1_brilho': 15, '0_1_impacto': 16, '0_2_wumpus': 17, '0_2_poco': 18, '0_2_ouro': 19, '0_2_parede': 20, '0_2_fedor': 21, '0_2_brisa': 22, '0_2_brilho': 23, '0_2_impacto': 24, '0_3_wumpus': 25, '0_3_poco': 26, '0_3_ouro': 27, '0_3_parede': 28, '0_3_fedor': 29, '0_3_brisa': 30, '0_3_brilho': 31, '0_3_impacto': 32, '0_4_wumpus': 33, '0_4_poco': 34, '0_4_ouro': 35, '0_4_parede': 36, '0_4_fedor': 37, '0_4_brisa': 38, '0_4_brilho': 39, '0_4_impacto': 40, '0_5_wumpus': 41, '0_5_poco': 42, '0_5_ouro': 43, '0_5_parede': 44, '0_5_fedor': 45, '0_5_brisa': 46, '0_5_brilho': 47, '0_5_impacto': 48, '1_0_wumpus': 49, '1_0_poco': 50, '1_0_ouro': 51, '1_0_parede': 52, '1_0_fedor': 53, '1_0_brisa': 54, '1_0_brilho': 55, '1_0_impacto': 56, '1_1_w

# Agente

In [8]:
class Agente():
  def __init__(self, mapa, solver, base_de_conhecimento):
    self.mapa = mapa
    self.solver = solver
    self.base_de_conhecimento = base_de_conhecimento
    self.percepcao = ()
    self.linha = 0
    self.coluna = 0
    self.orientacao = 0 # ângulo
    self.desempenho = 0
  
  def Agente(self):
    self.percepcao = self.receberSensor(self)
    self.inferirPercepcoes(self)
    self.escolherAcao(self) 
  
  @staticmethod
  def receberSensor(self):
    return self.mapa.retornarStatusDaPosica(self.linha, self.coluna)
  
  @staticmethod
  def inferirPercepcoes(self):
    # wumpus = self.percepcao[0][0]
    # poco = self.percepcao[0][1]
    # ouro = self.percepcao[0][2]
    fedor = self.percepcao[1][0]
    brisa = self.percepcao[1][1]
    brilho = self.percepcao[1][2]
    impacto = self.percepcao[1][3]
    grito = self.percepcao[1][4]

    '''
    if wumpus: self.solver.possuiWumpus(self.linha, self.coluna)
    else: self.solver.naoPossuiWumpus(self.linha, self.coluna)

    if poco: self.solver.possuiPoco(self.linha, self.coluna)
    else: self.solver.naoPossuiPoco(self.linha, self.coluna)

    if ouro: self.solver.possuiOuro(self.linha, self.coluna)
    else: self.solver.naoPossuiOuro(self.linha, self.coluna)
    '''

    if fedor == 1: self.solver.possui1Fedor(self.linha, self.coluna)
    elif fedor == 2: self.solver.possui2Fedor(self.linha, self.coluna)
    elif fedor == 3: self.solver.possui3Fedor(self.linha, self.coluna)
    else: self.solver.naoPossuiFedor(self.linha, self.coluna)

    if brisa == 1: self.solver.possui1Brisa(self.linha, self.coluna)
    elif brisa == 2: self.solver.possui2Brisa(self.linha, self.coluna)
    elif brisa == 3: self.solver.possui3Brisa(self.linha, self.coluna)
    else: self.solver.naoPossuiBrisa(self.linha, self.coluna)

    if impacto == 1: self.solver.possui1Impacto(self.linha, self.coluna)
    elif impacto == 2: self.solver.possui2Impacto(self.linha, self.coluna)
    else: self.solver.naoPossuiImpacto(self.linha, self.coluna)

    if brilho: self.solver.possuiBrilho(self.linha, self.coluna)
    else: self.solver.naoPossuiBrilho(self.linha, self.coluna)
  
  @staticmethod
  def escolherAcao(self):
    prioridade = ["pegar_ouro", "tatakai-tatakai-tatakai", "girar_aleatorio", "sair", "atirar"]
    acao = ""
    if self.perguntarOuro(self): 
      self.pegarOuro(self)
      acao = "pegar_ouro"
    elif self.peguntarAvancar(self): 
      self.avancar(self)
      acao = "tatakai"
    else: self.girarAleatorio(self)
    self.base_de_conhecimento.inserirAcao(acao)


  @staticmethod
  def perguntarOuro(self):
    return solver.perguntarRegra(f"{self.linha}_{self.coluna}_ouro")

  @staticmethod
  def pegarOuro(self):
    if mapa.pegarOuro(self.linha, self.coluna):
      self.ouro += 1
      self.desempenho += 1000
  
  @staticmethod
  def peguntarAvancar(self):
    linha, coluna = self.posicaoAFrente(self)
    try:
      return not (solver.perguntarRegra(f"{linha}_{coluna}_wumpus") or solver.perguntarRegra(f"{linha}_{coluna}_poco") or solver.perguntarRegra(f"{linha}_{coluna}_parede"))
    except:
      return False

  @staticmethod
  def posicaoAFrente(self):
    linha = self.linha
    coluna = self.coluna
    if self.orientacao == 0: coluna += 1
    elif self.orientacao == 90: linha -= 1
    elif self.orientacao == 180: coluna -= 1
    elif self.orientacao == 270: linha += 1
    return linha, coluna

  @staticmethod
  def avancar(self):
    if self.orientacao == 0: self.coluna += 1
    elif self.orientacao == 90: self.linha -= 1
    elif self.orientacao == 180: self.coluna -= 1
    elif self.orientacao == 270: self.linha += 1
    self.desempenho -= 1
  
  @staticmethod
  def girarAleatorio(self):
    direcao = ["esquerda", "direita"]
    direcao = choice(direcao)
    if direcao == "esquerda": self.girarParaEsquerda(self)
    elif direcao == "direita": self.girarParaDireita(self)
    acao = f"girar_{direcao}"
    self.base_de_conhecimento.inserirAcao(acao)

  @staticmethod
  def girarParaEsquerda(self):
    self.orientacao += 90
    if self.orientacao == 360: self.orientacao = 0
    self.desempenho -= 1
  
  @staticmethod
  def girarParaDireita(self):
    self.orientacao -= 90
    if self.orientacao == -90: self.orientacao = 270
    self.desempenho -= 1
  
  @staticmethod
  def sairDaCaverna(self):
    if self.linha == 1 and self.coluna == 1:
      self.desempenho -= 1
      # sair

# Main

In [9]:
mapa = Mapa(linhas = 4, colunas = 4, quantidade_wumpus = 1, quantidade_pocos = 2, quantidade_ouros = 2)
mapa.gerarMapa()
mapa.imprimirMapa()


parede           parede            parede    parede    parede   parede
parede       2impacto1brisa       1impacto  1impacto  2impacto  parede
parede        1impactopoco         1brisa      ok     1impacto  parede
parede  1impactoouro1brisabrilho     ok        ok     1impacto  parede
parede          2impacto          1impacto  1impacto  2impacto  parede
parede           parede            parede    parede    parede   parede



In [10]:
base_de_conhecimento = BaseDeConhecimento()
base_de_conhecimento.imprimirTabela()
solver = Solver(4, 4, base_de_conhecimento)
agente = Agente(mapa, solver, base_de_conhecimento)

 Tempo    Conhecimento    Raciocínio    Ação
-------  --------------  ------------  ------


In [11]:
agente.Agente()

In [12]:
mapa.imprimirMapa()
base_de_conhecimento.imprimirTabela()


parede           parede            parede    parede    parede   parede
parede       2impacto1brisa       1impacto  1impacto  2impacto  parede
parede        1impactopoco         1brisa      ok     1impacto  parede
parede  1impactoouro1brisabrilho     ok        ok     1impacto  parede
parede          2impacto          1impacto  1impacto  2impacto  parede
parede           parede            parede    parede    parede   parede

  Tempo   Conhecimento                       Raciocínio                            Ação
-------  --------------  ---------------------------------------------------  --------------
      1    -0_0_fedor
      2                                0_0_fedor, -1_0_wumpus
      3                                0_0_fedor, -0_1_wumpus
      4    0_0_brisa
      5                               falso, verdade, -1_0_poco
      6                               falso, verdade, -0_1_poco
      7                              falso, -1_0_poco, -0_1_poco
      8                        